# Core Analysis
Examine the core density matrix using the Complexity Reduction Framework.

In [ ]:
from os.path import join
geom = "renumber"
bsets = ["pcseg-0", "pcseg-1", "pcseg-2"]

markers = {}
msize = {}
colors = {}
markers["pcseg_0"] = "+"
markers["pcseg_1"] = "x"
markers["pcseg_2"] = "d"
markers["pcseg_3"] = "v"
msize["pcseg_0"] = 12
msize["pcseg_1"] = 12
msize["pcseg_2"] = 8
msize["pcseg_3"] = 8
colors["pcseg_0"] = "C0"
colors["pcseg_1"] = "C1"
colors["pcseg_2"] = "C2"
colors["pcseg_3"] = "C3"

In [ ]:
from pyntchem.io import read_pdb
with open(join("input", geom + ".pdb")) as ifile:
    sys = read_pdb(ifile)

Get number of electrons, core, virtual.

In [ ]:
nel = 0
ncore = 0
for frag in sys.values():
    for at in frag:
        nel += at.nel
        if at.sym != "H":
            ncore += 1

nocc = int(nel/2)
print(nocc, ncore)

## Calculations
Setup the calculations and run.

In [ ]:
from pyntchem.basis import BasisSet
basis = [BasisSet(x, sys.get_symlookup()) for x in bsets]

In [ ]:
from pyntchem.inputfile import Inputfile
inp = Inputfile()
inp.set_custom_dft({"pbe": 0.75}, {"pbe": 1.0}, 0.25)
inp.set_dft_prune_grid(50, 194)
inp.set_scf_guess("readdens")
inp["scf"].pulayperiod = 2
inp["scf"].facdamp = 0.95
inp["scf"].writeanal = True

inp["int2"].prelinkjthreshold = 1e-8
inp["int2"].prelinkkthreshold = 1e-4

In [ ]:
from pyntchem.calculator import JobscriptCalculator
calc = JobscriptCalculator(computer="Spring", skip=True, verbose=True)
args = {"nodes": 1, "tasks_per_node": 4, "omp": 9,  "queue": "winter2"}

Compute everything.

In [ ]:
from pyntchem.preprocessing import put_guess_matrix
from contextlib import suppress
from time import sleep
    
for b in basis:
    alp = join("work-basis", b.name, b.name + ".DensAlp.mtx")

    with suppress(IOError):
        put_guess_matrix(join("work-ch"), 
                         b.name, dens_alp_file=alp)

    calc.run(sys, inp, b, name=b.name, 
             run_dir=join("work-ch"),  **args)

while not calc.check_results(): sleep(10.0)
logfiles = {x: y.log for x, y in calc.calculations.items()}

## Analysis
Generate the required quantities.

In [ ]:
from pyntchem.postprocessing import NTChemTool
tool = NTChemTool()

Special versions that can manually accept a matrix and number of electron values.

In [ ]:
def run_compute_purity(self, sys, log, charges, kxs):
    from scipy.sparse import csc_matrix
    from scipy.io import mmread
    from pyntchem.postprocessing import _compute_purity_values

    fidx = self.get_frag_indices(sys, log)
    purity_array = _compute_purity_values(csc_matrix(kxs), charges, fidx)

    return purity_array

def fragment_bond_order(self, sys, fraglist1, fraglist2, log, kxs):
    from scipy.io import mmread
    from scipy.sparse import csr_matrix, csc_matrix
    from pyntchem.postprocessing import units_quadratic_traces

    frag_indices = self.get_frag_indices(sys, log)

    slist1 = sorted(fraglist1, key=lambda x: min(frag_indices[x]))
    slist2 = sorted(fraglist2, key=lambda x: min(frag_indices[x]))
    bond_orders = units_quadratic_traces(slist1, slist2, frag_indices,
                                         csc_matrix(kxs),
                                         csr_matrix(kxs), 0.5, 0.5)

    return bond_orders

Manually generated the density matrices P and PC.

In [ ]:
from scipy.io import mmread
from scipy.linalg import funm
from numpy import sqrt
from numpy import identity
from numpy.linalg import eigh

P = {}
PC = {}
for b in basis:
    S = mmread(logfiles[b.name].overlap).todense()
    H = mmread(logfiles[b.name].fockalp).todense()
    I = identity(S.shape[0])
    
    ISQ = funm(S, lambda x: -1.0/sqrt(x))
    WH = ISQ.dot(H).dot(ISQ)
    
    w, v = eigh(WH)

    P[b.name] = 2 * v[:, :nocc].dot(v[:, :nocc].T)
    PC[b.name] = 2 * v[:, :ncore].dot(v[:, :ncore].T)

Compare the sparsity of the two types of density matrices.

In [ ]:
def compute_sparsity(mat, thresh):
    k = 0
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if abs(mat[i, j]) < thresh:
                k += 1
    return 100*k / (mat.shape[0]*mat.shape[1])

In [ ]:
for b in basis:
    print(b.name, "%.2f"% compute_sparsity(P[b.name], 1e-5), "%.2f"% 
          compute_sparsity(PC[b.name], 1e-5))

We need these "charges" dictionaries to describe how many electrons are in a block.

In [ ]:
charges = {}
for fragid, frag in sys.items():
    charges[fragid] = sum(x.nel for x in frag)
    
core_charges = {}
for fragid, frag in sys.items():
    core_charges[fragid] = sum(1 for x in frag if x.sym != "H")

Now we're ready to compute the purities and bond orders.

In [ ]:
purities_occupied = {}
purities_core = {}

for b in basis:
    purities_occupied[b.name] = run_compute_purity(tool, sys, logfiles[b.name], charges, P[b.name])
    purities_core[b.name] = run_compute_purity(tool, sys, logfiles[b.name], core_charges, PC[b.name])

In [ ]:
bond_orders_occupied = {}
bond_orders_core = {}

for b in basis:
    log = logfiles[b.name]
    bond_orders_occupied[b.name] = fragment_bond_order(tool, sys, list(sys), list(sys), log, kxs=P[b.name])
    bond_orders_core[b.name] = fragment_bond_order(tool, sys, list(sys), list(sys), log, kxs=PC[b.name])

Order for plotting. We will use the BigDFT values to decide them.

In [ ]:
from pickle import load
with open("bigdft.cache", "rb") as ifile:
    bdftp, bdfto = load(ifile)

In [ ]:
order = sorted(sys, key=lambda x: int(x.split(":")[1]))

In [ ]:
relevant = []
for k1, v1 in bdfto.items():
    for k2, v2 in v1.items():
        if k1 == k2:
            continue
        if (k2, k1) in relevant:
            continue
        if "MOL" in k1:
            if v2 > 1e-2:
                relevant.append((k1, k2))
        else:
            if v2 > 5e-3:
                relevant.append((k1, k2))
relevant = sorted(relevant, key=lambda x: int(x[0].split(":")[1]))

Plot purities and bond orders.

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 2, figsize=(8, 5))

for key, val in purities_occupied.items():
    axs[0].plot([abs(val[k]) for k in order], 
                marker=markers[key], markersize=msize[key], label=key,
                color=colors[key])
    axs[0].plot([abs(purities_core[key][k]) for k in order], linestyle='--',
                marker=markers[key], markersize=msize[key], color=colors[key])
    
axs[0].set_yscale("log")
axs[0].set_xticks(range(len(order)))
axs[0].set_xticklabels(order, rotation=90)
axs[0].set_ylim(1e-6, 1e-1)
axs[0].tick_params(axis='both', which='major', labelsize=16)
axs[0].set_ylabel("Purity Indicator", fontsize=18)

for key, val in bond_orders_occupied.items():
    axs[1].plot([val[k[0]][k[1]] for k in relevant], 
             marker=markers[key], markersize=msize[key], label=key.upper().replace("_", "-"),
             color=colors[key])
    axs[1].plot([bond_orders_core[key][k[0]][k[1]] for k in relevant], linestyle='--',
             marker=markers[key], markersize=msize[key], color=colors[key])
    
axs[1].legend(loc="lower center", ncol=2, prop={'size': 11})
axs[1].set_ylim(5e-9, 1)
axs[1].set_xticks(range(len(relevant)))
axs[1].set_xticklabels(["-".join(x) for x in relevant], rotation=90)
axs[1].set_yscale("log")
axs[1].tick_params(axis='both', which='major', labelsize=16)

axs[1].set_ylabel("Fragment Bond Order", fontsize=18)
fig.tight_layout()
# fig.savefig("core_occ.png", dpi=600)

## Atomic Fragmentation
Look at the values using the atoms as their own fragments (remove hydrogens as well). For this we need a new system and charge values.

In [ ]:
from pyntchem.systems import System
from pyntchem.fragments import Fragment
asys = System()
i = 0
achrg = {}
achrgc = {}
for fragid in order:
    frag = sys[fragid]
    for at in frag:
        if at.sym == "H":
            continue
        asys["FRA:"+str(i)] = Fragment([at])
        achrg["FRA:"+str(i)] = at.nel
        achrgc["FRA:"+str(i)] = 2
        i += 1

Recompute purity.

In [ ]:
ap = {}
for b in basis:
    ap[b.name] = run_compute_purity(tool, asys, logfiles[b.name], achrg, P[b.name])
apc = {}
for b in basis:
    apc[b.name] = run_compute_purity(tool, asys, logfiles[b.name], achrgc, PC[b.name])

Plot.

In [ ]:
fig, axs = plt.subplots(1, 1)

for b in basis:
    axs.plot([abs(x) for x in ap[b.name].values()], 
             marker=markers[b.name], color=colors[b.name],
             markersize=msize[b.name], label=b.name.upper().replace("_", "-"))
    axs.plot([abs(x) for x in apc[b.name].values()], 
             marker=markers[b.name], color=colors[b.name],
             markersize=msize[b.name], linestyle="--")

axs.set_xticks(range(len(list(asys))))
axs.set_xticklabels([x[0].sym for x in asys.values()])
axs.tick_params(axis='both', which='major', labelsize=14)
axs.set_ylim(1e-4, 1)
axs.set_yscale("log")

axs.set_ylabel("Purity Indicator", fontsize=18)
axs.legend()
fig.tight_layout()
# fig.savefig("PI-Core.png", dpi=600)

## Eigenvalue Computations
We need a new atomic system that includes the hydrogens.

In [ ]:
asys2 = System()
core_charges2 = {}
i = 0
for frag in sys.values():
    for at in frag:
        asys2["FRA:"+str(i)] = Fragment([at])
        i += 1
        core_charges2["FRA:"+str(i)] = at.nel

Compute the eigenvalues (reference).

In [ ]:
envelopes = {}
envelopes["Core"] = (0, 29)
envelopes["Valence"] = (30, nocc)
envelopes["Virtual"] = (nocc, 323)

In [ ]:
w = {}
for b in basis:
    S = mmread(logfiles[b.name].overlap).todense()
    H = mmread(logfiles[b.name].fockalp).todense()
    
    ISQ = funm(S, lambda x: -1.0/sqrt(x))
    WH = ISQ.dot(H).dot(ISQ)
    
    w[b.name], _ = eigh(WH)
    
    # Basis, homo-lumo gap, core-valence gap
    print(b.name, 27.2114*(w[b.name][nocc] - w[b.name][nocc - 1]),
                  27.2114*(w[b.name][29] - w[b.name][29-1]))

Local in space approach without embedding first for fragments.

In [ ]:
w_block = {}
for b in basis:
    w_block[b.name] = []
    S = mmread(logfiles[b.name].overlap).todense()
    H = mmread(logfiles[b.name].fockalp).todense()
    log = logfiles[b.name]
    
    ISQ = funm(S, lambda x: -1.0/sqrt(x))
    WH = ISQ.dot(H).dot(ISQ)

    frag_indices = tool.get_frag_indices(sys, log)
    for fragid in sys:
        submat = WH[:,frag_indices[fragid]]
        submat = submat[frag_indices[fragid], :]
        
        eig, _ = eigh(submat)
        w_block[b.name].extend(eig)
    w_block[b.name] = sorted(w_block[b.name])

Then for the atoms.

In [ ]:
w_atm = {}
for b in basis:
    w_atm[b.name] = []
    S = mmread(logfiles[b.name].overlap).todense()
    H = mmread(logfiles[b.name].fockalp).todense()
    log = logfiles[b.name]
    
    ISQ = funm(S, lambda x: -1.0/sqrt(x))
    WH = ISQ.dot(H).dot(ISQ)

    frag_indices = tool.get_frag_indices(asys2, log)
    for fragid in asys2:
        submat = WH[:,frag_indices[fragid]]
        submat = submat[frag_indices[fragid], :]
        
        eig, _ = eigh(submat)
        w_atm[b.name].extend(eig)
    w_atm[b.name] = sorted(w_atm[b.name])

Plot errors.

In [ ]:
from numpy import array
fig, axs = plt.subplots(1, len(basis), figsize=(12, 6))
for i, b in enumerate(basis):
    data = []
    labels = []
    for e, r in envelopes.items():
        data.append([27.2114*abs(x - y) for x, y in zip(w_atm[b.name][r[0]:r[1]], w[b.name][r[0]:r[1]])])
        labels.append("ATOM-" + e)
    for e, r in envelopes.items():
        data.append([27.2114*abs(x - y) for x, y in zip(w_block[b.name][r[0]:r[1]], w[b.name][r[0]:r[1]])])
        labels.append("Fragment-" + e)
    axs[i].set_ylim(3e-4, 30)
        
    axs[i].boxplot(data)
    axs[i].set_yscale("log")
        
    axs[i].set_xticks(range(1, len(labels)+1))
    axs[i].set_xticklabels(labels, rotation=90)
    axs[i].tick_params(axis='both', which='major', labelsize=16)
    
axs[0].set_title("PCSEG-0", fontsize=18)
axs[1].set_title("PCSEG-1", fontsize=18)
axs[2].set_title("PCSEG-2", fontsize=18)
axs[0].set_ylabel("Error (eV)", fontsize=18)
fig.tight_layout()
# fig.savefig("orb-error.png", dpi=600)

Print out the nitrogen errors.

In [ ]:
for b in basis:
    print(b.name)
    print("Atomic & " + " & ".join(["%.2f"% (27.2114 * x) for x in w_atm[b.name][13:16]]) + "\\\\")
    print("Fragment & " + " & ".join(["%.2f"% (27.2114 * x) for x in w_block[b.name][13:16]]) + "\\\\")
    print("Full & " + " & ".join(["%.2f"% (27.2114 * x) for x in w[b.name][13:16]]) + "\\\\")

## Embedding Correction
We recompute these nitrogen values using the embedding technique.

In [ ]:
nfrags = [x for x in asys2 if asys2[x][0].sym == "N"]
embed = {}
threshold = 1e-4

for b in basis:
    print(b.name)
    embed[b.name] = {}
    for n in nfrags:
        bon = fragment_bond_order(tool, asys2, [n],
                                  [x for x in asys2 if x != n], 
                                  logfiles[b.name], kxs=PC[b.name])[n]
        embed[b.name][n] = []
        while sum(bon.values()) > threshold:
            maxv = max(bon, key=bon.get)
            _ = bon.pop(maxv)
            embed[b.name][n].append(maxv)
        print(n, embed[b.name][n], [asys2[at][0].sym for at in embed[b.name][n]])

Draw the embedding environment.

In [ ]:
from BigDFT.Interop.RDKitInterop import convert_system_to_rdkit, rdkit_visualize
from BigDFT.Fragments import Fragment
from IPython.display import SVG
from BigDFT.Visualization import get_colordict
from os import system
tempsys = System()
tempsys["TAR:0"] = Fragment()
tempsys["BUF:1"] = Fragment()
tempsys["OUT:2"] = Fragment()
for fragid in asys2:
    if fragid in ['FRA:25', 'FRA:24', 'FRA:6', 'FRA:26', 
                  'FRA:19', 'FRA:22', 'FRA:4', 'FRA:28', 'FRA:10']:
        tempsys["BUF:1"] += asys2[fragid]
    elif fragid == "FRA:21":
        tempsys["TAR:0"] = asys2[fragid]
    else:
        tempsys["OUT:2"] += asys2[fragid]
from pyntchem.io import write_pdb
with open("temp.pdb", "w") as ofile:
    write_pdb(tempsys, ofile)
system("obabel -ipdb temp.pdb -opdb > temp-con.pdb")
from BigDFT import IO
with open("temp-con.pdb", "r") as ifile:
    tempsys = IO.read_pdb(ifile)
rsys = convert_system_to_rdkit(tempsys)

colordict = {"TAR:0": (1,0.0,0), "BUF:1": (93/255,63/255,211/255), "OUT:2": (220/255,220/255,220/255)}
print(get_colordict(list(tempsys)))
SVG(rdkit_visualize(tempsys, format="SVG", colordict=colordict))
with open("buffer.svg", "w") as ofile:
    ofile.write(rdkit_visualize(tempsys, format="SVG", colordict=colordict))

Compute.

In [ ]:
from numpy import diag
for b in basis[:]:
    print(b.name)
    S = mmread(logfiles[b.name].overlap).todense()
    H = mmread(logfiles[b.name].fockalp).todense()
    log = logfiles[b.name]
    
    ISQ = funm(S, lambda x: -1.0/sqrt(x))
    WH = ISQ.dot(H).dot(ISQ)
    
    for n in nfrags:
        subsys = System()
        subsys[n] = asys2[n]
        for fid in embed[b.name][n]:
            subsys[fid] = asys2[fid]
        frag_indices = tool.get_frag_indices(subsys, log)
        sublen = len(frag_indices[n])
        
        subhmat = WH[:, sum(frag_indices.values(), [])]
        subhmat = subhmat[sum(frag_indices.values(), []), :]
        eigs, vecs = eigh(subhmat)
        
        vecs2 = PC[b.name][:, frag_indices[n]]
        vecs2 = vecs2[frag_indices[n], :]
        
        # we can use range sublen because we explicitly made the target the first fragment
        vecs = vecs[range(sublen), :]
        prod = vecs.T.dot(vecs2).dot(vecs)
        
        weights = diag(prod)
            
        idx = sorted(range(len(weights)), key=weights.__getitem__, reverse=True)[:sublen]
        pvals = sorted([27.2114*eigs[i] for i in idx])
        print(pvals[0])